## Sleep Recommendation Algorithm

**The algorithm will consist of three types of user preference**
1. The algorithm can help the user to determine what is the best bedtime, wake up time or duration based on their "sleep pattern" to improve their sleep quality(not directly associate it)
2. The algorithm can provide recommend range of bedtimes to the user for the given wake time
3. The algorithm give insights on their preferrable bedtime and wakeuptime and give an opinion on how they could change it

### Import Relevant Modules

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
from sklearn.ensemble import RandomForestRegressor

### Preprocess data

**If you want to add more fields you add it in this part or else you dont need to run this section of the code. It had already exported the file**

In [102]:
# Load Data from csv file
sleep_data = pd.read_csv('data/Sleep.csv')
servings1 = pd.read_csv('data/servings 1.csv', engine='python')
servings2 = pd.read_csv('data/servings 2.csv', engine='python')
servings3 = pd.read_csv('data/servings 3.csv', engine='python')

# Extract relevant Data from csv files
sleep_data = sleep_data[['date', 'Sleep Score', 'Total Bedtime', 'Bedtime Start', 'Bedtime End','Steps', 'HRV Balance Score']]
servings1 =  servings1[['Day', 'Alcohol (g)', 'Caffeine (mg)', 'Water (g)']]
servings2 =  servings2[['Day', 'Alcohol (g)', 'Caffeine (mg)', 'Water (g)']]
servings3 =  servings3[['Day', 'Alcohol (g)', 'Caffeine (mg)', 'Water (g)']]

#Combine Datas
servings1 = servings1.groupby('Day', as_index=False).sum()
servings2 = servings2.groupby('Day', as_index=False).sum()
servings3 = servings3.groupby('Day', as_index=False).sum()

# Add all servings files together and remove and day that are empty
servings_data = pd.concat([servings1, servings2, servings3])
sleep_data = sleep_data.dropna()
servings_data = servings_data.dropna()

# Rename the columns to better merge and visibility
sleep_data.rename(columns={'date': 'Date'}, inplace=True)
servings_data.rename(columns={'Day': 'Date'}, inplace=True)

sleep_data.rename(columns={'HRV Balance Score': 'Stress'}, inplace=True)
sleep_data.rename(columns={'Total Bedtime': 'Duration'}, inplace=True)
sleep_data.rename(columns={'Bedtime Start': 'Bedtime'}, inplace=True)
sleep_data.rename(columns={'Bedtime End': 'Waketime'}, inplace=True)
servings_data.rename(columns={'Alcohol (g)': 'Alcohol'}, inplace=True)
servings_data.rename(columns={'Caffeine (mg)': 'Caffeine'}, inplace=True)
servings_data.rename(columns={'Water (g)': 'Water'}, inplace=True)

# Convert Date to Date time(type) to be consistent
sleep_data['Date'] = pd.to_datetime(sleep_data['Date']).dt.date
servings_data['Date'] = pd.to_datetime(servings_data['Date']).dt.date

sleep_data['Bedtime'] = pd.to_datetime(sleep_data['Bedtime'], utc=True)
sleep_data['Waketime'] = pd.to_datetime(sleep_data['Waketime'], utc=True)

# Merge the sleep data and servings data
user_data = pd.merge(sleep_data, servings_data, on='Date')

# Clean unecessary Data
user_data["Stress"] = pd.to_numeric(user_data["Stress"], errors='coerce')
user_data = user_data.dropna()
user_data = user_data.reset_index(drop=True)


# Export Data to csv
user_data.to_csv("User Data.csv", index=False)


**Run the following snippet to get data**

only run one time

In [3]:
# Load User Data from csv
user_data = pd.read_csv('User Data.csv')

# Get user local time zone info
timezone = datetime.now().astimezone().tzinfo

# Convert time to local time zone for accuracy
user_data['Bedtime'] = pd.to_datetime(user_data['Bedtime']).dt.tz_convert(timezone).dt.round("min").dt.time
user_data['Waketime'] = pd.to_datetime(user_data['Waketime']).dt.tz_convert(timezone).dt.round("min").dt.time

# Convert Time to minutes
user_data['Bedtime'] = pd.to_timedelta(user_data['Bedtime'].astype(str)).dt.total_seconds() 
user_data['Waketime'] = pd.to_timedelta(user_data['Waketime'].astype(str)).dt.total_seconds() 

**Algorithm #1**

The first algorithm that it can work interchangeable for bedtime and waketime. It will recommend the user based on their prefer waketime and sleep quality(or max) the algorithm can recommend the user the best bedtime

In [67]:
# Load historical data
X = user_data[['Sleep Score', 'Duration', 'Waketime', 'Steps','Stress', 'Alcohol', 'Caffeine', 'Water']]
y = user_data['Bedtime']

In [ ]:
# Train the model
bedtime_model = RandomForestRegressor()
bedtime_model.fit(X.values,y)

# Calculate Ideal Bedtime
def calculate_ideal_bedtime(ideal_duration, ideal_sq, ideal_wakeup_time, steps, stress, alcohol, caffeine, water):

    # Change the unit to seconds to match the data unit
    ideal_duration = ideal_duration * 3600
    ideal_wakeup_time = timedelta(hours=ideal_wakeup_time.hour, minutes=ideal_wakeup_time.minute, seconds=ideal_wakeup_time.second).total_seconds()

    # Bedtime prediction pattern
    ideal_bedtime = bedtime_model.predict([[ideal_sq, ideal_duration, ideal_wakeup_time, steps, stress, alcohol, caffeine, water]])[0]

    # Convert to time
    ideal_bedtime = timedelta(seconds=ideal_bedtime)

    midnight = datetime.combine(datetime.today(), time.min)

    ideal_bedtime = midnight + ideal_bedtime

    return ideal_bedtime.time().replace(second=0, microsecond=0)

# User sample input
ideal_duration = 8 #hour
ideal_sq = 90
ideal_wakeup_time = time(7, 00, 0)
steps = 10000
stress = 80
alcohol = 0
caffeine = 200.0
water = 2000

ideal_bedtime = calculate_ideal_bedtime(ideal_duration, ideal_sq, ideal_wakeup_time, steps, stress, alcohol, caffeine, water)


In [79]:
print("Your ideal bedtime is: ", ideal_bedtime)

Your ideal bedtime is:  22:51:00


**Algorithm #2** 

The algorithm is giving the user a list of best bedtime based on what their range of prefered bed time is based on sleep quality

In [4]:
# Load historical data
X_2 = user_data[['Bedtime', 'Duration', 'Waketime', 'Steps','Stress', 'Alcohol', 'Caffeine', 'Water']]
y_2 = user_data['Sleep Score']

In [13]:
# Train the model
bedtime_model = RandomForestRegressor()
bedtime_model.fit(X_2.values,y_2)

# Calculate Ideal Bedtime
def calculate_ideal_bedtime_list(ideal_duration, ideal_wakeup_time, ideal_bedtime_start, ideal_bedtime_end, steps, stress, alcohol, caffeine, water):

    # Change the unit to seconds to match the data unit
    ideal_duration = ideal_duration * 3600
    ideal_wakeup_time = timedelta(hours=ideal_wakeup_time.hour, minutes=ideal_wakeup_time.minute, seconds=ideal_wakeup_time.second).total_seconds()

    ideal_bedtime_start = timedelta(hours=ideal_bedtime_start.hour, minutes=ideal_bedtime_start.minute, seconds=ideal_bedtime_start.second).total_seconds()
    ideal_bedtime_end = timedelta(hours=ideal_bedtime_end.hour, minutes=ideal_bedtime_end.minute, seconds=ideal_bedtime_end.second).total_seconds()
    
    # Store all the bed time
    ideal_bedtime_dict = {}
    bedtime = ideal_bedtime_start

    midnight = datetime.combine(datetime.today(), time.min)

    interval = 15 # minutes

    # Calculate the best bedtime and sort in dict
    while (bedtime <= ideal_bedtime_end):
        
        sleep_quality = bedtime_model.predict([[bedtime, ideal_duration, ideal_wakeup_time, steps, stress, alcohol, caffeine, water]])[0]

        # Convert to time
        ideal_bedtime = timedelta(seconds=bedtime)
        ideal_bedtime = midnight + ideal_bedtime
        ideal_bedtime = ideal_bedtime.time().replace(second=0, microsecond=0)

        # Store in dictionary
        ideal_bedtime_dict[ideal_bedtime] = sleep_quality
        
        # Increase Interval
        bedtime += interval * 60

    return ideal_bedtime_dict

# User sample input
ideal_duration = 8 #hour
ideal_sq = 90
ideal_wakeup_time = time(7, 00, 0)
steps = 10000
stress = 80
alcohol = 0
caffeine = 200.0
water = 2000
ideal_bedtime_start = time(20, 00, 0)
ideal_bedtime_end = time(23, 00, 0)

ideal_bedtime_dict = calculate_ideal_bedtime_list(ideal_duration, ideal_wakeup_time, ideal_bedtime_start, ideal_bedtime_end, steps, stress, alcohol, caffeine, water)


In [17]:
ideal_bedtime_dict = dict(sorted(ideal_bedtime_dict.items(), key=lambda x: x[1], reverse=True))

rank = 1
for k, v in ideal_bedtime_dict.items():
    print(f'{rank}. Bedtime: {k} Sleep quality: {v}')
    rank += 1

1. Bedtime: 22:00:00 Sleep quality: 88.14
2. Bedtime: 21:45:00 Sleep quality: 88.09
3. Bedtime: 20:30:00 Sleep quality: 87.72
4. Bedtime: 22:15:00 Sleep quality: 87.7
5. Bedtime: 22:30:00 Sleep quality: 87.69
6. Bedtime: 21:30:00 Sleep quality: 87.63
7. Bedtime: 22:45:00 Sleep quality: 87.51
8. Bedtime: 23:00:00 Sleep quality: 87.32
9. Bedtime: 20:45:00 Sleep quality: 87.31
10. Bedtime: 21:15:00 Sleep quality: 87.25
11. Bedtime: 20:15:00 Sleep quality: 87.04
12. Bedtime: 21:00:00 Sleep quality: 86.85
13. Bedtime: 20:00:00 Sleep quality: 86.71
